In [1]:
import pandas as pd
from sklearn.impute import KNNImputer

#{'feature_name': {'test_name' :{'success': [], 'details':[]}}
test_name =[]
feature_name = []
test_success = []
test_details = []
#report ={'feature_name': feature_name,'test_name': test_name,'success':test_success,'details':test_details}
report ={}


#Loading data from csv files
#Excel data files
def load_data(data_set):
    try:
        excel = ['.xls','.xlsx']
        if '.csv' in data_set:
            d1 = pd.read_csv(data_set,delimiter=',',header=0)
            print('CSV file loaded')
            print(d1.head())
        elif any(x in data_set for x in excel): 
            d1 = pd.read_excel(data_set,index_col=None)
            print('Excel file loaded')
            print(d1.head())
        return d1
    except Exception as err:
        print(err)
        print('Error encountered in loading data file')

    
#Missing values 

#Impute missing values with mean for Qualitative Data

def impute_missing_values_mean(data):
    try:
        if type(data) == pd.core.frame.DataFrame:
            for col in data.columns:
                if (data[col].isnull().sum()>0):
                    if any(data[col].dtype in x2 for x2 in  [x.value for x in QuantitativeDataType]):
                        data[col].fillna(value=data[col].mean(),inplace=True)
        else:
            data = data.fillna(data.mean())
        return data
    except Exception as err:
        print(err)
        print('Error encountered in loading data file')

 
 #Categorical Data
def impute_missing_values_mode(data):
    try:
        if type(data) == pd.core.frame.DataFrame:
            for col in data.columns:
                if (data[col].isnull().sum()>0):
                    if any(data[col].dtype in x2 for x2 in  [x.value for x in CategoricalDataType]):
                        print(col)
                        data[col].fillna(value=data[col].mode()[0],inplace=True)
                        
        else:
            data = data.fillna(data.value_counts().index[0])
            
        return data
    except Exception as err:
        print(err)
        print('Error encountered in imputing missing values - mode')
       


    # Impute missing values with KNN            

def impute_missing_values_knn(data,k=2):
    try:
        if type(data) == pd.core.frame.DataFrame:
            missing_cols = data.columns[data.isnull().any()]
            if len(missing_cols)>0:
                    imputer =KNNImputer(n_neighbors=k)
                    data = pd.DataFrame(imputer.fit_transform(data),columns=data.columns) 
        else:
            imputer =KNNImputer(n_neighbors=k)
            data =pd.DataFrame( (imputer.fit_transform(np.array(data).reshape(1,-1))).reshape(-1,1),columns=[data.name])                    

        return data  
    except Exception as err:
        print(err)
        print('Error encountered in imputing missing values - knn')

#Impute missing values with Interpolate
def impute_missing_values_interpolate(data):
    try:
        data_filled = data.interpolate()
            
        return data_filled
    except Exception as err:
        print(err)
        print('Error encountered in imputing missing values - interpolate')

In [2]:
import numpy as np
import enum
from enum import Enum, auto
import logging

# the use of Enum classes will prevent incorrect combination of values
class QuantitativeDataType(Enum):
    CONTINUOUS = [float, np.float64]
    DISCRETE = [int]

class CategoricalDataType(Enum):
    BOOLEAN = [bool]
    NUMERICAL = [float, int, np.float64, np.int64]
    CHARACTER = [str, object]
    
class DataType(Enum):
    """

    """
    QUANTITATIVE = [QuantitativeDataType.CONTINUOUS,
                   QuantitativeDataType.DISCRETE]
    CATEGORICAL = [CategoricalDataType.BOOLEAN,
                  CategoricalDataType.NUMERICAL,
                  CategoricalDataType.CHARACTER]
    #MISSING = 'MISSING'
    DATETIME = 'DATETIME'
    UNKNOWN = 'UNKNOWN'
    
class CustomWarning:
    
        def __init__(self, disclosure,level):
            self.level = level
            self.disclosure = disclosure

            
        def display(self,message, columns = ' ' ):
            
            logger = logging.getLogger('mylogger')
            logger.setLevel(self.level)

            handler = logging.FileHandler('mylog.log')
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            logger.addHandler(handler)
            
            if self.disclosure == 1:
                if self.level == 'CRITICAL':
                    logger.critical(message)
                elif self.level == 'WARNING':
                    logger.warning(message)
            elif self.disclosure == 2:
                if self.level == 'CRITICAL':
                    message = 'Critical Warning.' + message
                    logger.critical(message)
                elif self.level == 'WARNING':
                    message = 'Regular Warning.' + message
                    logger.warning(message)
            elif self.disclosure == 3:
                if self.level == 'CRITICAL':
                    message = 'Critical Warning. ' + message + 'Columns affected :' + columns
                    logger.critical(message)
                elif self.level == 'WARNING':
                    message = 'Regular Warning. ' + message  + 'Columns affected :' + columns
                    logger.warning(message) 
            print (message) 
            return message

In [3]:
import math #Checking if missing values exceed threshold limit(50%)
def check_missing_values_limit(data,percent,disclosure):
    MIN_NB_MISSING_DATA = math.ceil((percent/100)*data.shape[0])
    report_details = []
    for col in data.columns:
        elem_report ={}
       
        elem_report['feature_name']=col
        
        if (data[col].isnull().sum()>MIN_NB_MISSING_DATA):
            elem_report['success']='No'
            critical_warning = CustomWarning(disclosure,'CRITICAL')
            message = critical_warning.display(f'Missing value exceeds threshold limit {MIN_NB_MISSING_DATA}',col) 
            elem_report['msg']= message
        else:
            elem_report['success']='Yes'
            elem_report['msg']='Test passed'
        
        report_details.append(elem_report)
        print(report_details)
            
        report['check_missing_values_limit'] = report_details

    return report

In [4]:
#Checking samples limit

def check_number_of_samples(data,MIN_NB_SAMPLES,disclosure):
    
    sample_count = data.shape[0]
    elem_report ={}
    report_details = []
    elem_report['feature_name']='ALL'    
    
    if sample_count> MIN_NB_SAMPLES:
        elem_report['success']='No'
        critical_warning = CustomWarning(disclosure,'CRITICAL')
        message = critical_warning.display(f'Samples count exceeds the threshold limit {MIN_NB_SAMPLES}')
        elem_report['msg']= message
    else:
        elem_report['success']='Yes'
        elem_report['msg']='Test passed'
        
    report_details.append(elem_report)
    print(report_details)
        
    report['check_number_of_samples'] = report_details

    return report

In [ ]:
report_details

In [ ]:
csv_data1 = load_data('C:\\Users\\p.santosha.dasari\\Desktop\\Feature164\\pseudo_adni_mod.csv')

In [ ]:
import pandas as pd
def load_data(data_set):
    try:
        excel = ['.xls','.xlsx']
        if '.csv' in data_set:
            d1 = pd.read_csv(data_set,delimiter=';',header=0)
            print('CSV file loaded')
            print(d1.head())
        elif any(x in data_set for x in excel): 
            d1 = pd.read_excel(data_set,index_col=0)
            print('Excel file loaded')
            print(d1.head())
        return d1
    except:
        print('Exception encountered in loading data file')

In [5]:
d1 = load_data('/mnt/c/Users/p.santosha.dasari/Desktop/Feature164/Exceltest.xlsx')

Excel file loaded
   ID   Age Eligibility
0    1   45           Y
1    2   45           Y
2    3   33           N
3    4   54           Y
4    5   45           Y


In [6]:
a1 =check_missing_values_limit(d1,0.1,2)

[{'feature_name': 'ID ', 'success': 'Yes', 'msg': 'Test passed'}]
[{'feature_name': 'ID ', 'success': 'Yes', 'msg': 'Test passed'}, {'feature_name': 'Age', 'success': 'Yes', 'msg': 'Test passed'}]
Critical Warning.Missing value exceeds threshold limit 1
[{'feature_name': 'ID ', 'success': 'Yes', 'msg': 'Test passed'}, {'feature_name': 'Age', 'success': 'Yes', 'msg': 'Test passed'}, {'feature_name': 'Eligibility', 'success': 'No', 'msg': 'Critical Warning.Missing value exceeds threshold limit 1'}]


In [7]:
b1 = check_number_of_samples(d1,2,1)

Samples count exceeds the threshold limit 2
[{'feature_name': 'ALL', 'success': 'No', 'msg': 'Samples count exceeds the threshold limit 2'}]


In [8]:
b1

{'check_missing_values_limit': [{'feature_name': 'ID ',
   'success': 'Yes',
   'msg': 'Test passed'},
  {'feature_name': 'Age', 'success': 'Yes', 'msg': 'Test passed'},
  {'feature_name': 'Eligibility',
   'success': 'No',
   'msg': 'Critical Warning.Missing value exceeds threshold limit 1'}],
 'check_number_of_samples': [{'feature_name': 'ALL',
   'success': 'No',
   'msg': 'Samples count exceeds the threshold limit 2'}]}